# <center> Кластеризация изображений транспортных средств

## Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import MiniBatchKMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (silhouette_score, 
                             calinski_harabasz_score, 
                             davies_bouldin_score)

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings 
import pickle

from IPython.display import display, HTML

plt.style.use('bmh')
plt.rcParams["patch.force_edgecolor"] = True
warnings.simplefilter("ignore")

## 1. Знакомство со структурой данных

In [ ]:
desc_path = 'data/descriptors/'

effnet_data = pickle.load(open(desc_path+'efficientnet-b7.pickle', 'rb')) 
osnet_data = pickle.load(open(desc_path+'osnet.pickle', 'rb'))
vdc_color_data = pickle.load(open(desc_path+'vdc_color.pickle', 'rb'))
vdc_type_data = pickle.load(open(desc_path+'vdc_type.pickle', 'rb'))

print_data = lambda name, data: print(f'{name}:', '\n', 
                                      data, '\n', 
                                      f'{data.shape[0]} rows, {data.shape[1]} columns')

In [ ]:
print_data('EfficientNet', effnet_data)

In [ ]:
print_data('OSNet', osnet_data)

In [ ]:
print_data('VDC color regression', vdc_color_data)

In [ ]:
print_data('VDC type classification', vdc_type_data) 

> *Посмотрев на размерности каждой из заданных матриц, можно сказать, что нейросеть EfficientNet описывает изображения наиболее подробным образом - на 2560 дескрипторов. На мой взгляд, в конечном итоге, именно на дескрипторах этой модели кластеризация будет наиболее точной.*

**Примечание** Для удобства работы мы далее составим четыре DataFrame с путями до изображений и соответствующими им дескрипторами.

In [ ]:
img_paths = pd.read_csv('data/images_paths.csv')

img_paths.head()

## 2. Преобразование, очистка и анализ данных

Признаки, найденные с помощью некоторых моделей, исчисляются тысячами, что довольно много. Производить кластеризацию на таком большом количестве признаков, которые были сформированы исходными моделями глубокого обучения, довольно сложно и затратно по времени. 

Понизим размерность исходных дескрипторов с помощью соответствующих методов. Можно уменьшить размерность входных данных до 100 или 200 признаков — этого будет достаточно, чтобы произвести кластеризацию.

In [ ]:
RS = 12 # random_state

def standardise_and_decompose(data, n_components):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    
    decomposer = PCA(n_components=n_components, random_state=RS)
    data_decomposed = decomposer.fit_transform(data_scaled)
    
    return data_decomposed

In [ ]:
X_effnet = standardise_and_decompose(effnet_data, n_components=200) 
print_data('EfficientNet', X_effnet) 

In [ ]:
X_osnet = standardise_and_decompose(osnet_data, n_components=100)
print_data('OSNet', X_osnet) 

In [ ]:
X_vdc_color = standardise_and_decompose(vdc_color_data, n_components=50)
print_data('VDC color regression', X_vdc_color)

In [ ]:
X_vdc_type = standardise_and_decompose(vdc_type_data, n_components=100)
print_data('VDC type classification', X_vdc_type) 

## 3. Моделирование и оценка качества модели

После предобработки исходных данных произведите кластеризацию для каждого набора дескрипторов.

Для решения задачи используйте несколько различных методов, подобрав оптимальное количество кластеров для каждого метода и варианта дескрипторов.

В качестве метрики для подбора оптимального количества кластеров используйте внутренние меры:
* индекс Калински — Харабаса (`calinski_harabasz_score`) 
* индекс Дэвиса — Болдина (`davies_bouldin_score`)

### Кластеризация изображений (метод MiniBatchKMeans)

Поскольку исходных данных много, могут возникнуть проблемы с оперативной памятью и скоростью работы, например `K-means`, можно воспользоваться реализацией `MiniBatchKMeans`. 

In [ ]:
datasets = {'efficientnet': X_effnet, 
            'osnet': X_osnet, 
            'vdc_color_reg': X_vdc_color, 
            'vdc_type_cl': X_vdc_type}

clusters = list(range(2, 11))

chs_df = pd.DataFrame(index=clusters)
dbs_df = pd.DataFrame(index=clusters)


def get_clustering_scores(data, model, c_range):
    chs_list = []
    dbs_list = []
    
    for c in c_range:
        model_ = model(n_clusters=c, random_state=RS)
        model_.fit(ds)
        
        chs_list.append(calinski_harabasz_score(data, model_.labels_))
        dbs_list.append(davies_bouldin_score(data, model_.labels_))
    
    return chs_list, dbs_list


for name, ds in datasets.items():
    chs_df[name] = get_clustering_scores(ds, MiniBatchKMeans, clusters)[0]
    dbs_df[name] = get_clustering_scores(ds, MiniBatchKMeans, clusters)[1]

print('Calinski-Harabasz scores:')
display(chs_df)

print('Davies-Bouldin scores:')
display(dbs_df)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))

sns.lineplot(data=chs_df, markers=True, ax=ax[0])
sns.lineplot(data=dbs_df, markers=True, ax=ax[1])

ax[0].set_xlabel('clusters')
ax[0].set_title('calinski_harabasz_score')

ax[1].set_xlabel('clusters')
ax[1].set_title('davies_bouldin_score')

fig.show()

> *Поскольку индекс Калински-Харабаса нужно **максимизировать**, то самое оптимальное количество кластеров для всех дескрипторов - **2**. А индекс Девиса-Болдина нужно **минимизировать**, то это количество для всех разное, для EfficientNet и VDC type classification - **2**, для OSNet - **10**, и VDC color regression - **3**.*

> *Однако, данную визуализацию трудно интерпретировать в том смысле, что разные точки могут находиться очень близко друг к другу по оси Х и трудно разглядеть, какая выше, а какая ниже. Поэтому, чтобы знать наверняка, узнаем оптимальное количество кластеров с помощью сортировки значений получившихся таблиц.*

In [ ]:
def get_n_clusters(df, aggf):
    c_dict = {}
    
    for col in df.columns:
        if aggf == 'max':
            c_dict[col] = (df[col]
                           .sort_values(ascending=False)
                           .index[0])
        if aggf == 'min':
            c_dict[col] = (df[col]
                           .sort_values(ascending=True)
                           .index[0])
        
    return c_dict

print(get_n_clusters(chs_df, 'max'))
print(get_n_clusters(dbs_df, 'min'))

> *На мой взгляд, так определять оптимальное количество кластеров намного надежнее. Поскольку большая часть метрик показывают его как 2, то в конечном счете определим его именно так. Мне кажется, это связано с тем, что изображения автомобилей были сделаны либо спереди, либо сзади. Но мы проверим мою гипотезу на визуализации кластеров.* 

In [ ]:
mbkm = MiniBatchKMeans(n_clusters=2, random_state=RS)

print_series = lambda name, data: print(f'{name}:', '\n', 
                                        data, '\n', 
                                        f'{data.shape[0]} rows')

In [ ]:
y_effnet = mbkm.fit_predict(X_effnet)
print_series('EfficientNet labels', y_effnet) 

In [ ]:
y_osnet = mbkm.fit_predict(X_osnet)
print_series('OSNet labels', y_osnet)

In [ ]:
y_vdc_color = mbkm.fit_predict(X_vdc_color) 
print_series('VDC color regression labels', y_vdc_color)

In [ ]:
y_vdc_type = mbkm.fit_predict(X_vdc_type)
print_series('VDC type classification labels', y_vdc_type) 

### Интерпретация кластеров

In [ ]:
tsne_decomposer = TSNE(n_components=3, 
                       perplexity=100, 
                       n_iter=10000,
                       random_state=RS, 
                       n_jobs=-1,
                       verbose=1)

effnet_tsne = tsne_decomposer.fit_transform(X_effnet)

print_data('EfficientNet (TSNE-decomposed)', effnet_tsne)

In [ ]:
osnet_tsne = tsne_decomposer.fit_transform(osnet_decomposed)
vdc_color_tsne = tsne_decomposer.fit_transform(vdc_color_decomposed)
vdc_type_tsne = tsne_decomposer.fit_transform(vdc_type_decomposed)
#120

print_data('OSNet (TSNE-decomposed)', 
           osnet_tsne)
print_data('VDC color regression (TSNE-decomposed)', 
           vdc_color_tsne)
print_data('VDC type classification (TSNE-decomposed)', 
           vdc_type_tsne)  

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter()

In [ ]:
def plot_samples_images(data, cluster_label, nrows=3, ncols=3, figsize=(12, 5)):
    """Функция для визуализации нескольких 
       случайных изображений из кластера cluster_label.
       Пути до изображений и метки кластеров должны быть 
       представлены в виде DataFrame со столбцами "paths" и "cluster".

    Args:
        data (DataFrame): таблица с разметкой изображений и соответствующих им кластеров
        cluster_label (int): номер кластера изображений
        nrows (int, optional): количество изображений по строкам таблицы (по умолчанию 3)
        ncols (int, optional): количество изображений по столбцам (по умолчанию 3)
        figsize (tuple, optional): размер фигуры (по умолчанию (12, 5))
    """
    
    # Фильтруем данные по номеру кластера
    samples_indexes = np.array(data[data['cluster'] == cluster_label].index)
    # Перемешиваем результаты
    np.random.shuffle(samples_indexes)
    # Составляем пути до изображений
    paths = data.loc[samples_indexes, 'paths']

    # Создаём фигуру и набор координатных плоскостей
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    fig.suptitle(f"Images from cluster {cluster_label}", fontsize=16)
    
    # Создаём циклы по строкам и столбцамв таблице с координатными плоскостями
    for i in range(nrows):
        for j in range(ncols):
            # Определяем индекс пути до изображения
            path_idx = i * ncols + j
            if path_idx >= len(paths):
                break
                
            # Извлекаем путь до изображения
            path = paths.iloc[path_idx]
            
            # Читаем изображение
            img = plt.imread(path)
            
            # Отображаем его на соответствующей координатной плоскости
            axes[i,j].imshow(img)
            # Убираем пометки координатных осей
            axes[i,j].axis('off')